### LogRegress history based on all trials
- Miller et al. 2021

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import HistoryBasedLogisticModel
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

coef_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100)
    print(exp.sub_name)
    model = HistoryBasedLogisticModel(mab, n_past=10)
    model.fit()

    df = model.get_coef_df(form="long")
    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    coef_df.append(df)

coef_df = pd.concat(coef_df, ignore_index=True)
mab_subjects.GroupData().save(coef_df, "history_coef_alltrials_anirudh")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
coef_df = grpdata.history_coef_alltrials_anirudh

coef_names = ["reward seeking", "choice preservation", "effect of outcome"]
for c, coef_name in enumerate(coef_names):

    df = coef_df[coef_df["coef_name"] == coef_name]

    ax = fig.subplot(fig.gs[c])
    ax.axhline(0, color="gray", lw=0.8, zorder=0)

    plot_kw = dict(
        data=df,
        x="past_id",
        y="coef",
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
    )
    sns.stripplot(
        # palette=["#5040BF", "#AFBF40"],
        palette=["gray", "gray"],
        size=3,
        alpha=0.5,
        dodge=True,
        **plot_kw,
    )
    sns.pointplot(
        dodge=0.4,
        palette="husl",
        linestyle="none",
        markersize=4,
        errorbar="se",
        err_kws={"linewidth": 0.95},
        **plot_kw,
    )

    orders = np.arange(1, 11)
    pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_title(coef_name)
    ax.legend("")

### LogRegress histroy based on first 100 trials


In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import HistoryBasedLogisticModel
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

coef_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    model = HistoryBasedLogisticModel(mab, n_past=10)
    model.fit()

    df = model.get_coef_df(form="long")
    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    coef_df.append(df)

coef_df = pd.concat(coef_df, ignore_index=True)
mab_subjects.GroupData().save(coef_df, "history_coef_100trials_anirudh")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
coef_df = grpdata.history_coef_anirudh

coef_names = ["reward seeking", "choice preservation", "effect of outcome"]
for c, coef_name in enumerate(coef_names):

    df = coef_df[coef_df["coef_name"] == coef_name]

    ax = fig.subplot(fig.gs[c])
    ax.axhline(0, color="gray", lw=0.8, zorder=0)

    plot_kw = dict(
        data=df,
        x="past_id",
        y="coef",
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
    )
    sns.stripplot(
        # palette=["#5040BF", "#AFBF40"],
        palette=["gray", "gray"],
        size=3,
        alpha=0.5,
        dodge=True,
        **plot_kw,
    )
    sns.pointplot(
        dodge=0.4,
        palette="husl",
        linestyle="none",
        markersize=4,
        errorbar="se",
        err_kws={"linewidth": 0.95},
        **plot_kw,
    )

    orders = np.arange(1, 11)
    pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="Levene", **stat_kw, color="k", verbose=True)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_title(coef_name)
    ax.legend("")

### Estimating Qlearning parameters (2 $\alpha$ model)
- Estimate learning parameter for chosen and unchosen arms/choices. If the animals are learning, then one should expect that the chosen and unchosen arm will have postive and negative $\alpha$ respectively.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab)
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "beta"],
            "param_values": np.array([qlearn.alpha_c, qlearn.alpha_u, qlearn.beta]),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_params_anirudh")

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns

params_df = mab_subjects.GroupData().qlearning_2alpha_params_anirudh

fig = plotting.Fig(1, 2, size=(4, 3), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp")
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)


alpha_df = params_df[params_df["param"] != "beta"]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

beta_df = params_df[params_df["param"] == "beta"]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### Qlearning with perseverance and scaler params
- When we estimated alpha parameters for chosen and unchosen choices, we found that unstructured env had higher alpha values for chosen arms compared to structured env. So we asked if 'persevrance' for arms/choices is making alpha_chosen higher for unstructured environment.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab, model="persev")
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_persev_anirudh")

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

params_df = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp", hue_order=["unstruc", "struc"])
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

orders = ["alpha_chosen", "alpha_unchosen", "persev"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

orders = ["scaler", "beta"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### Params estimation for "Structured" sessions in Unstructured Environment.
- Basically select sessions where the probabilities sum up to 1 i.e, correlated sessions withing Unstructured environment. Estimate various params like alpha_chosen, alpha_unchosen etc. on these sessions and compare these with Structured Environment.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    session_prob_sum = mab.probs[mab.is_session_start.astype(bool)].sum(axis=1)
    good_sessions = mab.sessions[session_prob_sum == 1]
    assert np.all(good_sessions)

    mab = mab.keep_sessions_by_id(good_sessions)

    qlearn = QlearningEstimator(mab, model="persev")
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(
    params_df, "qlearning_2alpha_persev_correlated_within_unstructured_anirudh"
)

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

df1 = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh
df2 = (
    mab_subjects.GroupData().qlearning_2alpha_persev_correlated_within_unstructured_anirudh
)
df2["grp"] = "struc_in_unstruc"
params_df = pd.concat([df1, df2], ignore_index=True)

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(
    x="param",
    y="param_values",
    hue="grp",
    hue_order=["unstruc", "struc", "struc_in_unstruc"],
)
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

# orders = ["alpha_chosen", "alpha_unchosen", "persev"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

# orders = ["scaler", "beta"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")